In [2]:


import astropy
print('astropy',astropy.__version__ )
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes
from astropy.coordinates import SkyCoord
from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
import numpy as np                          
import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy
import astropy.io.fits as fits                                                          
from astropy.wcs import WCS                 
from astropy import units as u              
import pyspeckit as psk   
import astrodendro
import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%matplotlib inline

#%matplotlib widget

astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16


In [2]:
files = pylab.array(['CMZ_3mm_HCO+.fits','CMZ_3mm_HCN.fits','CMZ_3mm_H13CN.fits','CMZ_3mm_H13CO+.fits' ])


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file).unmasked_copy()    
    #header = fits.getheader(file)
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    subcube = sc[:,:,:] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1000. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        HCO_Plus = sc_K_kms.unmasked_copy()
        HCO_Plus.write('cmz_HCOp.cube.K-kms.pbcor.fits',overwrite=True) 
        
    if i==1:
        HCN = sc_K_kms.unmasked_copy()

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
HCN.allow_huge_operations=True

HCOPH = HCO_Plus.header


del HCO_Plus #rem from mem


HCN_reproject = HCN.reproject(HCOPH,use_memmap=True)  
del HCN #rem from mem
del HCOPH

#HCN_reproject_interp = HCN_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
#Veloxcities already matched
#del HCN_reproject

HCN_reproject.write('cmz_HCN.cube.reproject.K-kms.pbcor.fits',overwrite=True)
#del HCN_reproject_interp         
print("done")        

done


In [3]:
files = pylab.array(['CMZ_3mm_HCO+.fits','CMZ_3mm_HCN.fits','CMZ_3mm_H13CN.fits','CMZ_3mm_H13CO+.fits' ])


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file).unmasked_copy()    
    #header = fits.getheader(file)
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    subcube = sc[:,:,:] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1000. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        HCO_Plus = sc_K_kms.unmasked_copy()
        #HCO_Plus.write('cmz_HCN.cube.reproject.K-kms.pbcor.fits',overwrite=True)
        
    if i==2:
        
        H13CN = sc_K_kms.unmasked_copy()

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
#del scW
H13CN.allow_huge_operations=True

HCOPH = HCO_Plus.header


del HCO_Plus #rem from mem


H13CN_reproject = H13CN.reproject(HCOPH,use_memmap=True)  
del H13CN #rem from mem
del HCOPH

#H13CN_reproject_interp = H13CN_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
#del H13CN_reproject
H13CN_reproject.write('cmz_H13CN.cube.reproject.K-kms.pbcor.fits',overwrite=True)        
#del H13CN_reproject_interp         
print("done")        

done


In [4]:
files = pylab.array(['CMZ_3mm_HCO+.fits','CMZ_3mm_HCN.fits','CMZ_3mm_H13CN.fits','CMZ_3mm_H13CO+.fits' ])


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file).unmasked_copy()    
    #header = fits.getheader(file)
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    subcube = sc[:,:,:] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1000. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        HCO_Plus = sc_K_kms.unmasked_copy()
        #HCO_Plus.write('cmz_HCN.cube.reproject.K-kms.pbcor.fits',overwrite=True)
        
    if i==3:
        
        H13COp= sc_K_kms.unmasked_copy()

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
#del scW
H13COp.allow_huge_operations=True

HCOPH = HCO_Plus.header


del HCO_Plus #rem from mem


H13COp_reproject = H13COp.reproject(HCOPH,use_memmap=True)  
del H13COp #rem from mem
del HCOPH

#H13COp_reproject_interp = H13COp_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
#del H13COp_reproject
H13COp_reproject.write('cmz_H13COp.cube.reproject.K-kms.pbcor.fits',overwrite=True)        
#del H13CN_reproject_interp         
print("done")        

done


In [ ]:
files = pylab.array(['CMZ_3mm_HCO+.fits','CMZ_3mm_HCN.fits','CMZ_3mm_H13CN.fits','CMZ_3mm_H13CO+.fits' ])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
        
    if i==5:
        NGC_H13CN_J1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_H13CN_J1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

NGC_H13CN_J1_reproject = NGC_H13CN_J1.reproject(HCOPH,use_memmap=True)  
del NGC_H13CN_J1 #rem from mem
del HCOPH
NGC_H13CN_J1_reproject_interp = NGC_H13CN_J1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_H13CN_J1_reproject
NGC_H13CN_J1_reproject_interp.write()#('NGC_H13CN_J1_0.cube.reproject.K-kms.pbcor.fits',overwrite=True)  
del NGC_H13CN_J1_reproject_interp        
    
     
print("done")             

In [7]:
files = pylab.array(['CMZ_3mm_HCO+.fits','CMZ_3mm_HCN.fits','CMZ_3mm_H13CN.fits','CMZ_3mm_H13CO+.fits' ])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==6:
        NGC_CO_J3_2 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_CO_J3_2.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

NGC_CO_J3_2_reproject = NGC_CO_J3_2.reproject(HCOPH,use_memmap=True)  
del NGC_CO_J3_2 #rem from mem
del HCOPH

NGC_CO_J3_2_reproject_interp = NGC_CO_J3_2_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_CO_J3_2_reproject
NGC_CO_J3_2_reproject_interp.write()#('NGC_CO_J3_2.cube.reproject.K-kms.pbcor.fits',overwrite=True)      
del NGC_CO_J3_2_reproject_interp  

print("done")         

done


In [8]:
files = pylab.array(['CMZ_3mm_HCO+.fits','CMZ_3mm_HCN.fits','CMZ_3mm_H13CN.fits','CMZ_3mm_H13CO+.fits' ])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==7:
        CCH_N1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
CCH_N1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

CCH_N1_reproject =CCH_N1.reproject(HCOPH,use_memmap=True)  
del CCH_N1 #rem from mem
del HCOPH
CCH_N1_reproject_interp = CCH_N1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del CCH_N1_reproject
CCH_N1_reproject_interp.write()#('CCH_N1.cube.reproject.K-kms.pbcor.fits',overwrite=True)   
del CCH_N1_reproject_interp         
print("done")            

done


In [2]:


files = pylab.array(['CMZ_3mm_HCO+.fits','gc_ATLASGAL_850micron_dust.fits'])
      



for i,file in enumerate(files):
    
    if i==0:
        subcube = SpectralCube.read(file) 
    
    
    if i==1:
        subcube = spectral_cube.Projection.from_hdu(fits.open(file)[0])
    #print(subcube.hdu.header)
    
    
    #Put in the right system
    
    subcube.allow_huge_operations=True
    
    
    
    if i==0:
        vel, dec, ra = subcube.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = subcube.moment0()
    if i==1:
        Cont = subcube
        

        
del dec
del ra
Cont.allow_huge_operations=True
#HCOPH = NGC_HCO_Plus.header
HCOPH = NGC_HCO_Plus.hdu.header
#HCOPH['NAXIS'] = 2
print(HCOPH)
#print([HCOPH['NAXIS{0}'.format(i + 1)] for i in range(HCOPH['NAXIS'])][::-1])
print([Cont.hdu.header['NAXIS{0}'.format(i + 1)] for i in range(Cont.hdu.header['NAXIS'])][::-1])
print(np.shape(NGC_HCO_Plus))
del NGC_HCO_Plus #rem from mem

Cont_reproject =Cont.reproject(HCOPH)  
del HCOPH

Cont_reproject.write('CMZ_Continuum_Reproject.fits',overwrite=True) 
print(np.shape(Cont_reproject))
del Cont_reproject          
print("done")        

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -32 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  765                                                  NAXIS2  =                  165                                                  BLANK   =                   -1  /                                               BUNIT   = 'K m s-1 '                                                            CELLSCAL= 'CONSTANT'  /                                                         RESTFREQ=    8.91885260000E+10  /                                               BMAJ    =    1.08300000429E-02  /                                               BMIN    =    1.08300000429E-02  /                                               DATAMIN =   -1.53687677383E+01  /                                               DATAMAX =    3.78722572327E+00  /       

In [ ]:

#Left Half of the CMZ
files = pylab.array(['CMZ_3mm_HCO+.fits',"ADP.2016-01-20T15:35:07.077.fits"])
      



for i,file in enumerate(files):
    
    if i==0:
        subcube = SpectralCube.read(file) 
    
    
    if i==1:
        subcube = spectral_cube.Projection.from_hdu(fits.open(file)[0])
    #print(subcube.hdu.header)
    
    
    #Put in the right system
    
    subcube.allow_huge_operations=True
    
    
    
    if i==0:
        vel, dec, ra = subcube.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = subcube.moment0()
    if i==1:
        Cont = subcube
        

        
del dec
del ra
Cont.allow_huge_operations=True
#HCOPH = NGC_HCO_Plus.header
HCOPH = NGC_HCO_Plus.hdu.header
#HCOPH['NAXIS'] = 2
print(HCOPH)
#print([HCOPH['NAXIS{0}'.format(i + 1)] for i in range(HCOPH['NAXIS'])][::-1])
print([Cont.hdu.header['NAXIS{0}'.format(i + 1)] for i in range(Cont.hdu.header['NAXIS'])][::-1])
print(np.shape(NGC_HCO_Plus))
del NGC_HCO_Plus #rem from mem

Cont_reproject =Cont.reproject(HCOPH)  
del HCOPH

Cont_reproject.write('CMZ_Continuum_Reproject_Left_Half.fits',overwrite=True) 
print(np.shape(Cont_reproject))
del Cont_reproject          
print("done")        

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -32 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  765                                                  NAXIS2  =                  165                                                  BLANK   =                   -1  /                                               BUNIT   = 'K m s-1 '                                                            CELLSCAL= 'CONSTANT'  /                                                         RESTFREQ=    8.91885260000E+10  /                                               BMAJ    =    1.08300000429E-02  /                                               BMIN    =    1.08300000429E-02  /                                               DATAMIN =   -1.53687677383E+01  /                                               DATAMAX =    3.78722572327E+00  /       

In [4]:
#Right
files = pylab.array(['CMZ_3mm_HCO+.fits',"12CO_GC_359-000_mosaic.fit"])


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file).unmasked_copy()    
    #header = fits.getheader(file)
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    subcube = sc[:,:,:] # Make a subcube
    
    if i==1:
        lll=0
        #subcube=subcube*u.K #The CO obs are not in units, but unit K is stated in the header
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1000. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True 
    print(sc_kms[0][0])
    sc_K_kms = sc_slab#.to(u.K)
    
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
    print(vel)
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        HCO_Plus = sc_K_kms.unmasked_copy()
        HCO_Plus.write('cmz_HCOp.cube.K-kms.pbcor.fits',overwrite=True) 
        
    if i==1:
        CO32 = sc_K_kms.unmasked_copy()

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
CO32.allow_huge_operations=True

HCOPH = HCO_Plus.header
print(HCOPH,CO32.header)
HCOPH['CTYPE3'] = 'VRAD'#Change from VRAD-W2F to VRAD, since it means sampled in wave, expressed in freq, same as VRAD, see: https://docs.astropy.org/en/stable/_modules/astropy/wcs/wcsapi/fitswcs.html?highlight=VRAD#

del HCO_Plus #rem from mem


CO32_reproject = CO32.reproject(HCOPH,use_memmap=True)  
del CO32 #rem from mem
del HCOPH

#HCN_reproject_interp = HCN_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
#Veloxcities already matched
#del HCN_reproject

CO32_reproject.write('CMZ_CO_3_2_Right.cube.reproject.K-kms.pbcor.fits',overwrite=True)
#del HCN_reproject_interp         
print("done")        

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan na

KeyboardInterrupt: 

In [3]:
#Left
files = pylab.array(['CMZ_3mm_HCO+.fits',"12CO_GC_001-002_mosaic.fit"])


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file).unmasked_copy()    
    #header = fits.getheader(file)
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    subcube = sc[:,:,:] # Make a subcube
    
    if i==1:
        lll=0
        #subcube=subcube*u.K #The CO obs are not in units, but unit K is stated in the header
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1000. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True 
    print(sc_kms[0][0])
    sc_K_kms = sc_slab#.to(u.K)
    
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
    print(vel)
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        HCO_Plus = sc_K_kms.unmasked_copy()
        HCO_Plus.write('cmz_HCOp.cube.K-kms.pbcor.fits',overwrite=True) 
        
    if i==1:
        CO32 = sc_K_kms.unmasked_copy()

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
CO32.allow_huge_operations=True

HCOPH = HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'#Change from VRAD-W2F to VRAD, since it means sampled in wave, expressed in freq, same as VRAD, see: https://docs.astropy.org/en/stable/_modules/astropy/wcs/wcsapi/fitswcs.html?highlight=VRAD#

del HCO_Plus #rem from mem


CO32_reproject = CO32.reproject(HCOPH,use_memmap=True)  
del CO32 #rem from mem
del HCOPH

#HCN_reproject_interp = HCN_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
#Veloxcities already matched
#del HCN_reproject

CO32_reproject.write('CMZ_CO_3_2_Left.cube.reproject.K-kms.pbcor.fits',overwrite=True)
#del HCN_reproject_interp         
print("done")        

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan na